In [1]:
import json
import os
import cv2
import shutil
import random

In [2]:
image_file = os.listdir('images')
json_files = os.listdir('labels')
shutil.rmtree('COCO_style/images/all')
shutil.rmtree('COCO_style/labels/all')
os.mkdir('COCO_style/images/all')
os.mkdir('COCO_style/labels/all')
for json_file in json_files:
    with open('labels/'+json_file,'r') as f_json:
        f = json.load(f_json)
        image_name = f['imagePath'].split('\\')[-1]
        if image_name not in image_file:
            print(image_name + ' not exist!')
            continue
        image = cv2.imread('images/'+image_name)
        height, width, _ = image.shape
        image_id = image_name.split('.')[0]
        plate = f['shapes'][0]['label']
        plate = plate.replace(' ','')
        plate = plate.replace('-','')
        x1 = int(f['shapes'][0]['points'][0][0])
        y1 = int(f['shapes'][0]['points'][0][1])
        x2 = int(f['shapes'][0]['points'][1][0])
        y2 = int(f['shapes'][0]['points'][1][1])
        x_center, y_center = (x2+x1)/2/width, (y2+y1)/2/height
        x_width, y_height  = (x2-x1)/width,   (y2-y1)/height
        label_txt = '0 ' + str(x_center) + ' ' + str(y_center) + ' ' + str(x_width) + ' ' + str(y_height)
        # print(image_name,plate,x1,y1,x2,y2)
        cv2.imwrite('COCO_style/images/all/'+plate+'.png', image)
        with open('COCO_style/labels/all/' + plate+'.txt','w') as f_txt:
            f_txt.write(label_txt)

In [3]:
image_ids = os.listdir('COCO_style/labels/all')
image_ids = [_.split('.')[0] for _ in image_ids]
random.shuffle(image_ids)
train_num = len(image_ids)*9//10
shutil.rmtree('COCO_style/images/train')
shutil.rmtree('COCO_style/images/val')
shutil.rmtree('COCO_style/labels/train')
shutil.rmtree('COCO_style/labels/val')
os.mkdir('COCO_style/images/train')
os.mkdir('COCO_style/images/val')
os.mkdir('COCO_style/labels/train')
os.mkdir('COCO_style/labels/val')

for image_id in image_ids[:train_num]:
    shutil.copy('COCO_style/images/all/'+image_id+'.png', 'COCO_style/images/train')
    shutil.copy('COCO_style/labels/all/'+image_id+'.txt', 'COCO_style/labels/train')
for image_id in image_ids[train_num:]:
    shutil.copy('COCO_style/images/all/'+image_id+'.png', 'COCO_style/images/val')
    shutil.copy('COCO_style/labels/all/'+image_id+'.txt', 'COCO_style/labels/val')

In [4]:
train_num

1736